# 05.6 RNN - Ejercicio splicing para Entrega

__Ejercicio de redes neuronales para estudiar de secuencias de RNA__


In [1]:
import numpy as np
import pandas as pd
from IPython import display

Con las herramientas de sk-learn se reproducirá la red diseñada para el problema de RNA splicing por los autores:

M. O. Noordewier and G. G. Towell and J. W. Shavlik Training Knowledge-Based Neural Networks to Recognize Genes in DNA Sequences. Advances in Neural Information Processing Systems, vol. 3, Morgan Kaufmann (1991)

<img src="images/kbann-splicing.png" width="500px">

## Carga de datos a partir de CSV

Como el archivo no incluye los nombres de las columnas se asigna el nombre a partir de una lista con el parámetro __names__ de __read_csv__

In [2]:
df = pd.read_csv('data/splice.txt')
df.head()

,Class,Instance,Sequence
0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...
1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...
2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...
3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...
4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...


Todas las secuencias tienen 60 pares de bases. La primera columna (__Class__) indica la clase a la que pertenece la secuencia: EI para las que proporcionan un salto exon-intrón (donores); IE para las secuencias que contienen una frontera intron-exón (aceptores); N para aquellas secuencias que no son ni EI ni IE. 

La segunda columna (__Instance__) es una etiqueta para cada una de las instancias (filas), y la tercera columna da la __secuencia__ correspondiente en terminos del alfabeto A, T, C y G.

## Preproceso de las secuencias

Algunas de las secuencias tienen ambiguedad en determinadas posiciones, de forma que incluyen las letras D, N, S o R de acuerdo con la siguiente tabla:

| Letra | Significado |
| ----- | ------- |
| D | A o G o T |
| N | A o G o T o C |
| S | C o G |
| R | A o G |

Estos caracteres ambiguos aparecen en muy pocas instancias, así que se van a eliminar de los datos para simplificar el analisis. Primero se realiza un proceso de filtrado para saber cuantas secuencias quedan:

__Se borran las secuencias ambiguas como ya se vió en el ejercicio anterior__

In [3]:
df = df[~df['Sequence'].str.contains("D|N|S|R")]
df

,Class,Instance,Sequence
0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...
1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...
2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...
3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...
4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...
...,...,...,...
3185,N,ORAHBPSBD-NEG-2881,TCTCTTCCCTTCCCCTCTCTCTTTCTTTCTTTTCTCTCCTCTTCTC...
3186,N,ORAINVOL-NEG-2161,GAGCTCCCAGAGCAGCAAGAGGGCCAGCTGAAGCACCTGGAGAAGC...
3187,N,ORARGIT-NEG-241,TCTCGGGGGCGGCCGGCGCGGCGGGGAGCGGTCCCCGGCCGCGGCC...
3188,N,TARHBB-NEG-541,ATTCTACTTAGTAAACATAATTTCTTGTGCTAGATAACCAAATTAA...


### Primero se prepara la matriz X de los conjuntos de entranamiento y pruebas

Se separa la secuencia en 60 campos de un caracter identificando la base

In [4]:
from sklearn.model_selection import train_test_split
X, y = df.values[:,2], df.values[:,0]
X_train, X_test, y_train, y_test =train_test_split(X, y, train_size=0.7, random_state=0, stratify=y, shuffle=True)

In [5]:
X_60 = [[letra for letra in fila] for fila in X_train]
y_real = y_train

In [6]:
X_train60 = [[letra for letra in fila] for fila in X_train]
X_test60 = [[letra for letra in fila] for fila in X_test]
print(len(X_train60), len(X_train60[0]), len(y_train))
print(len(X_test60), len(X_test60[0]), len(y_test))

2222 60 2222
953 60 953


In [7]:
dfw = pd.DataFrame(X_train60)
dfw.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,G,A,T,G,T,T,A,A,C,C,...,A,A,T,C,A,G,C,C,C,T
1,G,T,C,T,T,T,G,C,T,T,...,C,A,G,G,C,G,A,T,G,G
2,C,C,T,G,T,C,T,C,C,T,...,G,G,G,C,C,G,A,T,T,T
3,C,T,A,C,A,G,C,A,G,C,...,G,T,C,C,T,G,T,C,C,A
4,T,G,G,A,G,A,C,T,C,A,...,A,C,T,A,T,G,C,C,G,G


In [8]:
df = pd.DataFrame(X_train60)
bases=['A','T','C','G']
base_mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
for iCol in range(60):
    df[iCol] = df[iCol].map(base_mapping)
X_train60 = df.values

df = pd.DataFrame(X_test60)
bases=['A','T','C','G']
base_mapping = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
for iCol in range(60):
    df[iCol] = df[iCol].map(base_mapping)
X_test60 = df.values

X_train60.shape, X_test60.shape

((2222, 60), (953, 60))

## Efectuar el entrenamiento con el Perceptron multicapa (Multilayer perceptron - MLP)

La librería __sk-learn__ en el módulo __neural_network__ implementa el perceptron multicapa en la clas __MLPClassifier__.

Los parámetros del constructor de la clase __MLPClassifier__ son:
- __hidden_layer_sizes__ : Este parámetro permite establecer el número de capas y el número de nodos que se desean tener en el clasificador de redes neuronales. Cada elemento de la tupla representa el número de nodos en la i-ésima posición, donde i es el índice de la tupla. Por tanto, la longitud de la tupla denota el número total de capas ocultas en la red.
- __max_iter__ : número de épocas de entrenamiento.
- __activation__ : función de activación de las capas ocultas.
- __solver__ : algoritmo empleado en la optimización de los pesos de los nodos.
- __random_state__ : establece una semilla para reproducir los mismos resultados.

In [9]:
len(X_train60[0]), np.unique(y_train), len(X_train60), len(y_train), len(X_test60), len(y_test)

(60, array(['EI', 'IE', 'N'], dtype=object), 2222, 2222, 953, 953)

In [11]:
# Creamos objeto con el constructor
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=3000,random_state=1,hidden_layer_sizes=(240, 43))

In [12]:
## Entrenamiento de la red
mlp.fit(X_train60, y_train)

MLPClassifier(hidden_layer_sizes=(240, 43), max_iter=3000, random_state=1)

__Se presenta la exactitud del conjunto de entrenamiento y pruebas__

In [13]:
print("Exactitud del conjunto de entrenamiento: {:.3f}".format(mlp.score(X_train60, y_train)))
print("Exactitud del conjunto de prueba: {:.3f}".format(mlp.score(X_test60, y_test)))

Exactitud del conjunto de entrenamiento: 1.000
Exactitud del conjunto de prueba: 0.855
